In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler # Pure FP16 Imports
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from tqdm import tqdm
import numpy as np
import copy
import os
import gc
import sys

# Add parent directory to path for imports (notebook runs from notebooks/ directory)
sys.path.append(os.path.dirname(os.getcwd()))
from src.data.data_loader import load_data

# --- HYPERPARAMETERS ---
MODEL_NAME = "microsoft/deberta-v3-large" 
MAX_LEN = 350       
BATCH_SIZE = 8    # Kept small (4) because DeBERTa-Large is heavy even with FP16
EPOCHS = 10
LR = 1e-4           
WEIGHT_DECAY = 0.1  
LORA_DROPOUT = 0.1
# Model will be saved to models/ directory in project root
MODEL_PATH = os.path.join("..", "models", "autojudge_best_acc_model.pth")
PATIENCE = 3

# Create models directory if it doesn't exist
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True) 

# --- MODEL DEFINITION ---
class AutoJudge(nn.Module):
    def __init__(self, model_name, freeze_bert=False):
        super(AutoJudge, self).__init__()
        
        # Load Model (Safetensors to fix security error)
        # Gradient Checkpointing REMOVED
        self.bert = AutoModel.from_pretrained(model_name, use_safetensors=True)
        
        self.bert_dim = self.bert.config.hidden_size 
        
        self.processor = nn.Sequential(
            nn.Linear(self.bert_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        self.classifier = nn.Linear(256, 3) 
        self.regressor = nn.Sequential(
            nn.Linear(256, 1),
            nn.Sigmoid() 
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_emb = outputs.last_hidden_state[:, 0, :]
        x = self.processor(cls_emb)
        
        logits_cls = self.classifier(x)
        raw_score = self.regressor(x)
        final_score = raw_score * 9.0 + 1.0 
        
        return logits_cls, final_score

class TaskDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.texts = df['full_text'].tolist()
        self.y_cls = df['label_cls'].tolist()
        self.y_score = df['label_score'].tolist()
        self.tokenizer = tokenizer

    def __len__(self): return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]),
            max_length=MAX_LEN,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            'ids': enc['input_ids'].squeeze(0),
            'mask': enc['attention_mask'].squeeze(0),
            'y_cls': torch.tensor(self.y_cls[idx], dtype=torch.long),
            'y_score': torch.tensor(self.y_score[idx], dtype=torch.float)
        }

def evaluate(model, loader, device, criterion_cls, criterion_reg):
    model.eval()
    all_preds_cls = []
    all_targets_cls = []
    all_preds_score = []
    all_targets_score = []
    total_loss = 0
    
    with torch.no_grad():
        # Use Autocast in eval too for speed/memory
        with autocast():
            for batch in tqdm(loader, desc="Evaluating", leave=False):
                ids = batch['ids'].to(device)
                mask = batch['mask'].to(device)
                y_c = batch['y_cls'].to(device)
                y_s = batch['y_score'].to(device)
                
                logits, scores = model(ids, mask)
                
                loss = criterion_cls(logits, y_c) + criterion_reg(scores.squeeze(), y_s)
                total_loss += loss.item()
                
                preds_c = torch.argmax(logits, dim=1)
                all_preds_cls.extend(preds_c.cpu().numpy())
                all_targets_cls.extend(y_c.cpu().numpy())
                
                all_preds_score.extend(scores.squeeze().cpu().numpy())
                all_targets_score.extend(y_s.cpu().numpy())
            
    avg_loss = total_loss / len(loader)
    acc = accuracy_score(all_targets_cls, all_preds_cls)
    mae = mean_absolute_error(all_targets_score, all_preds_score)
    rmse = np.sqrt(mean_squared_error(all_targets_score, all_preds_score))
    
    return avg_loss, acc, mae, rmse

def train_and_eval():
    torch.cuda.empty_cache()
    gc.collect()
    
    df = load_data()
    if df is None: return
    
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    print(f"📊 Dataset Split: {len(train_df)} Training | {len(test_df)} Testing")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🚀 Device: {device} | Model: {MODEL_NAME}")
    
    # Tokenizer fix
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
    
    train_ds = TaskDataset(train_df, tokenizer)
    test_ds = TaskDataset(test_df, tokenizer)
    
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
    
    model = AutoJudge(MODEL_NAME, freeze_bert=False)
    
    peft_config = LoraConfig(
        r=16, 
        lora_alpha=32, 
        target_modules=["query_proj", "key_proj", "value_proj", "dense"], 
        lora_dropout=LORA_DROPOUT,
        bias="none"
    )
    
    model.bert = get_peft_model(model.bert, peft_config)
    model.to(device)
    
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    
    # FP16 Scaler
    scaler = GradScaler()
    
    total_steps = len(train_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=int(0.1 * total_steps), 
        num_training_steps=total_steps
    )
    
    loss_fn_cls = nn.CrossEntropyLoss()
    loss_fn_reg = nn.MSELoss()
    
    best_acc = 0.0
    early_stop_counter = 0
    best_model_state = None
    
    print("\n🏋️ Starting Training (Pure FP16 Mode)...")
    for epoch in range(EPOCHS):
        model.train()
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
        
        for batch in loop:
            ids = batch['ids'].to(device)
            mask = batch['mask'].to(device)
            y_c = batch['y_cls'].to(device)
            y_s = batch['y_score'].to(device).unsqueeze(1)
            
            optimizer.zero_grad()
            
            # --- FP16 CONTEXT ---
            with autocast():
                pred_c, pred_s = model(ids, mask)
                loss = loss_fn_cls(pred_c, y_c) + loss_fn_reg(pred_s, y_s)
            
            # Scale Loss -> Backward -> Step
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            scheduler.step()
            loop.set_postfix(loss=loss.item())
        
        # Validation
        val_loss, acc, mae, rmse = evaluate(model, test_loader, device, loss_fn_cls, loss_fn_reg)
        print(f"   📉 Val Loss: {val_loss:.4f} | 🏆 Acc: {acc*100:.2f}% | MAE: {mae:.2f}")
        
        if acc > best_acc:
            best_acc = acc
            early_stop_counter = 0
            best_model_state = copy.deepcopy(model.state_dict())
            torch.save(best_model_state, MODEL_PATH)
            print(f"   ✅ New Best Accuracy! Model saved.")
        else:
            early_stop_counter += 1
            print(f"   ⚠️ Accuracy didn't improve ({early_stop_counter}/{PATIENCE})")
            
        if early_stop_counter >= PATIENCE:
            print(f"\n⛔ Early stopping triggered after {epoch+1} epochs.")
            break

    print(f"\n🔄 Loading model with Highest Accuracy ({best_acc*100:.2f}%)...")
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    final_loss, final_acc, final_mae, final_rmse = evaluate(model, test_loader, device, loss_fn_cls, loss_fn_reg)
    
    print("\n" + "="*30)
    print("📢 FINAL EVALUATION REPORT (Best Accuracy Model)")
    print("="*30)
    print(f"✅ Classification Accuracy : {final_acc*100:.2f}%")
    print(f"✅ Regression MAE (Score)  : {final_mae:.4f}")
    print(f"✅ Regression RMSE (Score) : {final_rmse:.4f}")
    print("="*30 + "\n")

if __name__ == "__main__":
    train_and_eval()